In [4]:
import torch
from Network import GraphNet

The number of nodes is the only parameter with no default, so the graph must be created before the network.

In [6]:
adjacency_matrix = torch.ones((5, 5))
adjacency_matrix[0, 1], adjacency_matrix[1, 0] = 0, 0
adjacency_matrix[0, 3], adjacency_matrix[3, 0] = 0, 0
adjacency_matrix[0, 4], adjacency_matrix[4, 0] = 0, 0

Network is created as so:

In [7]:
net = GraphNet(n_nodes=adjacency_matrix.size()[0])

The input to the network is a vector of length 2 * (n_nodes ^ 2) + n_nodes. The first n_nodes^2 is the adjacency matrix, the next is the clustering matrix specifying which nodes are in the same cluster and the final n_nodes indicate which node was most recently placed into a cluster.

In [8]:
clustering_list = torch.tensor([1, 1, 2, 3, 0])
clustering_mat = net.get_clustering_matrix(clustering_list, 4)
last_node_placed = torch.zeros(5)
last_node_placed[3] = 1
b = torch.concat((torch.tensor(adjacency_matrix).flatten(), torch.tensor(clustering_mat).flatten(), last_node_placed))

C:\Users\toell\AppData\Local\Temp\ipykernel_9980\2140478734.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.concat((torch.tensor(adjacency_matrix).flatten(), torch.tensor(clustering_mat).flatten(), last_node_placed))


To train, one must first sample a number of clusterings using sample_forward, which will sample using the current network along with added randomness. The amount of randomness is set using the parameter 'gamma' when instantiating the network. Default is 0.5

In [10]:
X = net.sample_forward(adjacency_matrix=adjacency_matrix)
net.train(X, epochs=10)

100%|██████████| 10/10 [00:30<00:00,  3.02s/it]


Once finished, sampling again will produce different results. If you only want complete clusterings, set the net property 'termination_chance' to 0.

In [13]:
net.termination_chance = 0
X1 = net.sample_forward(adjacency_matrix)

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\toell\OneDrive\Documents\GitHub\GFlowNetsA\GraphClustering\Network\Network.py:234: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  clustering_list[node_chosen] = torch.tensor(cluster_index_chosen + 1, dtype=torch.float32)
100%|██████████| 100/100 [00:06<00:00, 14.65it/s]


These new samples have the same length as the input to the network, so use indexing to get the clustering matris or use the method of the GraphNEt class called get_matrices_from_state

In [14]:
clusterings = []
for x in X:
    clusterings.append(net.get_matrices_from_state(x))